In [1]:
import time
import numpy as np
import pinocchio as pin
import crocoddyl

from py_biconvex_mpc.motion_planner.biconvex import BiConvexMP

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator

from cnt_plan_utils import SoloCntGen


solo import 


In [2]:
robot = Solo12Config.buildRobotWrapper()
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
n_eff = 4
m = pin.computeTotalMass(robot.model)
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

In [4]:
# initial and ter state
X_init = np.zeros(9)
X_init[0:3] = q0[0:3]
X_ter = X_init.copy()

# contact plan
st = 0.2 # step time 
sh = 0.1 # step height
sl = np.array([0.1,0.0,0]) # step length
n_steps = 6 # number of steps
T = st*(n_steps + 2)
dt = 5e-2

X_ter[0:3] += sl*(n_steps)
X_nom = np.zeros((9*int(T/dt)))
X_nom[2::9] = X_init[2]

cnt_planner = SoloCntGen(T, dt, 1)
# print(cnt_plan)

In [7]:
# weights
W_X = np.array([1e-5, 1e-5, 1e-5, 1e-4, 1e-4, 1e-4, 3e3, 3e3, 3e3])
W_X_ter = 10*np.array([1e+5, 1e+5, 1e+5, 1e+5, 1e+5, 1e+5, 1e+5, 1e+5, 1e+5])

W_F = np.array(4*[1e+1, 1e+1, 1e+1])

rho = 1e+5 # penalty on dynamic constraint violation

# constraints 
bx = 0.25
by = 0.25
bz = 0.25
fx_max = 15
fy_max = 15
fz_max = 15

# optimization
mp = BiConvexMP(m, dt, T, n_eff, rho = rho)
mp.create_contact_array(cnt_plan)
mp.create_bound_constraints(bx, by, bz, fx_max, fy_max, fz_max)
mp.create_cost_X(W_X, W_X_ter, X_ter, X_nom)
mp.create_cost_F(W_F)
com_opt, F_opt, mom_opt = mp.optimize(X_init, 30)

iter number 0
1.8266978437979784
iter number 1
0.6317075373983148
iter number 2
0.8242191217105466
iter number 3
0.2824243677484799
iter number 4
0.08342925853134164
iter number 5
0.07381220746374344
iter number 6
0.06273169240365951
iter number 7
0.05118628781711644
iter number 8
0.040075045011105405
iter number 9
0.03189839819066453
iter number 10
0.028768434001397995
iter number 11
0.029376195849224594
iter number 12
0.031624343410218275
iter number 13
0.0324935566261324
iter number 14
0.031991048494855935
iter number 15
0.03012830179439071
iter number 16
0.025301085512603926
iter number 17
0.022106280470451337
iter number 18
0.02112045811636421
iter number 19
0.01861055056735656
iter number 20
0.01666693682652152
iter number 21
0.014977292975713615
iter number 22
0.013020041330842608
iter number 23
0.010924166651857536
iter number 24
0.008001654692138975
iter number 25
0.006205433046599641
iter number 26
0.005031547276051113
iter number 27
0.004356880552822022
iter number 28
0.0042

In [38]:
cnt_planner.reset(T, dt)
cnt_plan = cnt_planner.create_trot_plan(st, sl, n_steps)
ik_solver = cnt_planner.create_ik_step_costs(cnt_plan, sh, [1e+2, 1e+4])

In [39]:
ik_solver.ik.add_com_position_tracking_task(0, T, com_opt, 1e+4, "com_track_cost")
ik_solver.create_centroidal_task(mom_opt, 0, T, "mom_track_cost", 1e+4)
xs = ik_solver.optimize(x0, wt_xreg=5e-3)

In [40]:
for i in range(len(xs)):
    time.sleep(0.2)
    viz.display(xs[i][:robot.model.nq])

In [10]:
viz.viewer.jupyter_cell()

In [41]:
for i in range(len(xs)):
    q = xs[i][:robot.model.nq]
    v = xs[i][robot.model.nq:]
    pin.forwardKinematics(robot.model, robot.data, q, v)
    pin.updateFramePlacements(robot.model, robot.data)
#     print(pin.centerOfMass(robot.model, robot.data, q, v)[2])
    pin.computeCentroidalMomentum(robot.model, robot.data)
    print(np.round(robot.data.hg, 2)[4], np.round(mom_opt[i][4],2))
    

0.0 -0.0
0.02 -0.02
0.04 -0.04
0.05 -0.05
-0.07 -0.06
-0.04 -0.05
-0.06 -0.04
-0.02 -0.01
0.09 0.04
0.01 0.0
-0.02 -0.01
-0.05 -0.0
0.06 0.04
0.01 -0.01
-0.01 -0.02
0.01 -0.01
0.09 0.03
0.05 -0.01
0.0 -0.03
0.0 -0.01
0.08 0.03
0.02 -0.02
0.02 -0.03
-0.03 -0.02
0.09 0.02
0.01 -0.02
0.01 -0.03
0.02 -0.02
0.02 0.01
0.01 -0.04
0.0 -0.05
0.0 -0.03
-0.0 0.0


In [16]:
j = 0
tmp = com_opt[0][j]
err = 0
for i in range(1, len(com_opt)):
    tmp += dt*mom_opt[i][j]/m
    err += abs(com_opt[i][j] - tmp)/len(com_opt)

print(err)

0.020552451498495713
